# <u> Problema de los ascensores</u> (Versión Paralelizada)

#### G05 - Boris Carballa Corredoira, Juan Carlos Villanueva Quirós, Francisco Javier Blázquez Martínez

Solución final del problema paralelizado, el edificio consta de varios bloques con varios ascensores, donde los ascensores son considerados como elementos independientes capaces de ejecutar sus acciones simultáneamente. En esta versión, añadimos en los comentarios los cambios con respecto a la versión anterior (<u>estos estarán subrayados</u>).

<u>Para paralelizar, la idea fundamental será añadir el tiempo de llegada en el estado, tanto para las personas como para los ascensores. Esto nos permitirá ir marcando el momento en el que sucede cada acción y cambiar el punto de vista: buscar no ya la solución en la que los ascensores se desplazan menos plantas, sino aquella en la que es mínimo el tiempo de la persona que más tarda en llegar a su destino.</u> Lo explicamos en detalle a continuación:

In [70]:
from search import *
import copy
import time

#### Constantes (las mismas que en la versión secuencial)
+ *NUM_PLANTAS*: Numero de plantas del problema
+ *NUM_ASCENSORES*: Numero de ascensores a tener en cuenta
+ *NUM_PERSONAS*: Numero de personas a considerar. Se necesita para que <u>los *SolutionManager*, secuencial y paralelo,</u> muestren correctamente la solucion con varios ascensores
+ *CAP_ASCENSOR*: Capacidad de los ascensores
+ *EN_ASCENSOR*: Nos indica que una persona se encuentra dentro de un ascensor
+ *NINGUNA*: Usada para indicar que una lista es vacia
+ *EN_DESTINO*: Denota que una persona ha llegado a su destino

In [71]:
NUM_PLANTAS    = 12 #Para rango por defecto del ascensor
NUM_ASCENSORES = 4
NUM_PERSONAS = 5 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 2

EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

#### Solution Manager
Importamos el notebook *SolutionManager* <u>y *SolutionManagerParalelo*</u> en el cual tenemos almacenados todos los procedimientos que se ejecutan una vez calculada la solución para mejorar la visualización de la misma.
Nótese que ahora también necesitamos una versión paralelo ya que el formato de las acciones que imprimiremos será una versión ampliada.

In [72]:
%run SolutionManager.ipynb
%run SolutionManagerParalelo.ipynb

#### Clase NodeState

Clase que representa un nodo del espacio de exploración. El estado de dicho nodo se compone de:

 + *personas*: Lista con la planta en la que se encuentra cada persona, si una persona se encuentra dentro del ascensor toma el  valor de la constante EN_ASCENSOR, si se encuentra en su destino toma el valor de la costante EN_DESTINO
 + <u>*personas_tiempo_llegada*</u>: Lista con el instante en el que la persona i-ésima llegó a su planta, guardada en personas[i]
 + *ascensor_pos*: Lista con las plantas en la que se encuentra cada ascensor
 + *ascensor_personas*: Lista de listas con las personas (índices de la lista "personas") que se encuentran dentro de cada ascensor
 + *ascensor_rango*: Lista de tuplas (x,y) que índican el rango de cada ascensor, donde x es la planta mínima e y la planta máxima. Si no se introduce ningún rango, suponemos por defecto que pueden ir a todas las plantas
 + <u>*ascensores_tiempo_llegada*</u>: Lista con el instante en el que el ascensor i-ésimo llegó a su planta actual, almacenada en ascensor_pos[i]
 
Por otro lado, hemos definido los operadores igual (**eq**), menor que (**lt**) y **hash** para permitir la comparación entre nodos y posibilitar así la correcta inserción de estos en estructuras de datos para que las búsquedas usen control de repetidos.

In [73]:
class NodeState:

    def __init__(self, init_personas, ascensor_rango=None):
        self.personas = init_personas
        self.personas_tiempo_llegada = [0 for _ in range(NUM_PERSONAS)]
        self.ascensor_personas = [[] for _ in range(NUM_ASCENSORES)]
        self.ascensores_tiempo_llegada = [0 for _ in range(NUM_ASCENSORES)]
        if ascensor_rango is None:
            self.ascensor_rango = [(0,NUM_PLANTAS) for _ in range(NUM_ASCENSORES)] #[min,max], te deja lo mas arriba/abajo que puede
            self.ascensor_pos = [0 for _ in range(NUM_ASCENSORES)]
        else:
            self.ascensor_rango = ascensor_rango
            self.ascensor_pos = [rango[0] for rango in ascensor_rango]
        
    def __eq__(self, nodo):
        return self.personas == nodo.personas and self.ascensor_pos == nodo.ascensor_pos and self.ascensor_personas == nodo.ascensor_personas and self.ascensores_tiempo_llegada==nodo.ascensores_tiempo_llegada and self.personas_tiempo_llegada==nodo.personas_tiempo_llegada
   
    def __hash__(self):
        return hash((tuple(self.personas), tuple(self.personas_tiempo_llegada), tuple(self.ascensor_pos), tuple(self.ascensores_tiempo_llegada), tuple(tuple(x) for x in self.ascensor_personas)))
    
    def __lt__(self,nodo):
        return True

#### Clase AscensoresParalelizados
Clase que representa el problema en versión final paralelizada. <u> Result, path_cost y la heurística son las funciones que han cambiado.</u> Consta de varias funciones:

+ **Init**: Dado un nodo inicial y un nodo objetivo, inicializa la clase

    + *initial*: nodo desde el que empezamos a buscar <u>con los rangos de cada ascensor</u>
    + *goal*: Plantas a las que quieren llegar las respectivas personas
    + *analizados*: número de nodos analizados hasta el momento

+ **Actions**: Dado un nodo, devuelve las acciones posibles que podemos ejecutar desde dicho nodo. Las acciones posibles son: subir, bajar,coger_persona y dejar_persona. En esta versión, calculamos las acciones posibles para cada ascensor.
 
    + *Codificación Acciones*. Hemos decidido codificar las acciones posibles de la siguiente manera:
        + (subir, i, j) &rArr; El ascensor i sube j plantas
        + (bajar, i, j) &rArr; El ascensor i baja j plantas
        + (coger_persona, i, j) &rArr; El ascensor i coge a la persona j
        + (dejar_persona, i, j) &rArr; El ascesnro i deja a la persona j
      
    + *Variables locales*:
        + personas_esta_planta_suben: Lista de las personas que se encuentran en la planta del ascensor y su planta objetivo está por encima
        + personas_esta_planta_bajan: Lista de las personas que se encuentran en la planta del ascensor y su planta objetivo está por debajo
        + persona_menos_arriba: Menor planta de las personas que se encuentran por encima del ascensor y dentro de su rango
        + persona_menos_abajo: Mayor planta de las personas que se encuentran por debajo del ascensor y dentro de su rango
        + destino_menos_arriba: Menor destino de las personas que se encuentran dentro del ascensor o bien el límite máximo del rango si todos los destinos están por encima de este
        + destino_menos_abajo: Mayor destino de las personas que se encuentran dentro del ascensor o bien el límite mínimo del rango si todos los destinos están por debajo de este
        
    + *Restricciones*. Hemos decidido implementar una serie de restricciones basadas en invariantes implicitos del problema que nos permiten no dar una acción cuando sea innecesario (no nos aporta un camino hacia la solución óptima) eliminando ramas sin perder optimalidad. Todas estas restricciones han sido probadas válidas en las hipótesis del problema (de las cuales cabe destacar la no penalización por parada de los ascensores) por el método de reducción de diferencias:
        1. Un ascensor nunca tiene dentro personas que quieran subir y bajar. O bien tiene dentro a gente que quiere bajar, o bien a gente que quiere subir. De esta forma, evitamos alejar de su objetivo innecesariamente a las personas que quieren bajar cuando el ascensor sube y viceversa
        2. Si alguien llega a su planta objetivo, forzamos a que esta baje del ascensor, es decir, devolvemos como única acción posible dejar a dicha persona. Siempre será más óptimo dejar a la persona una vez alcance su objetivo y no alejarla de este
        3. Solo subimos cuando haya una persona esperando en una planta por encima del ascensor o cuando haya alguna persona dentro del ascensor cuya planta objetivo esté por encima. De esta manera, nos evitamos dar la opción de subir cuando no aporte nada al problema. (Respectivamente con bajar)
        4. No subimos cuando haya un hueco en el ascensor y alguien en la planta que quiera subir. De esta forma, obligamos a coger a esa persona antes de subir pues siempre será más eficiente acercarla si es posible. (Respectivamente con bajar)

+ **Result**. Dado un nodo y una acción, devuelve el resultado de aplicar al nodo dicha acción.<u> También involucra actualizar como es debido las nuevas variables de marcaje temporal de los distintos elementos</u>

+ **Goal_test**. Dado un nodo, comprueba si hemos alcanzado la solución. Habremos alcanzado el objetivo cuando todas las personas tengan el valor EN_DESTINO

+ **path_cost**. <u> Mientras que en las versiones anteriores el coste era el número de plantas que se hacía subir o bajar al ascensor sobre el que se ejecutaba la acción, ahora, al contar con la ejecución paralela, únicamente tenemos en cuenta al ascensor más lento. El método "path_cost" tiene en cuenta los tiempos de todos los ascensores, no únicamente del que ejecuta la acción. Así, acciones como *subir* y *bajar* un determinado ascensor puede tener coste cero siempre y cuando sea plenamente paralelizable. </u>

+ **h**. Heurística a usar en la búsqueda *A\**. Hemos elegido para esta primera versión que sea la suma para cada persona de las diferencias entre la planta en la que se encuentra y la planta objetivo dividido esto entre la constante CAP_ASCENSOR<u>*NUM_ASCENSORES, ya que ahora pueden moverse varios ascensores a la vez</u>. La heurística es <u>admisible</u>. No dividir por esta constante implica usar una heurística no admisible (y por tanto no llegar a solución óptima necesariamente) pero obtener soluciones en un tiempo de ejecución menor.

In [74]:
class AscensoresParalelizados(Problem) :    
   
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal
        self.analizados = 0

    def actions(self, state):
        
        accs = list()
        
        for num_ascensor in range(0,NUM_ASCENSORES):
            # Parámetros relativos al ascensor a evaluar
            ascensor_pos      = state.ascensor_pos[num_ascensor]     # Planta del ascensor en cuestión
            ascensor_rango    = state.ascensor_rango[num_ascensor]
            ascensor_personas = state.ascensor_personas[num_ascensor] # Personas dentro del ascensor a tratar
            personas = state.personas
            
            ascensor_vacio    = not ascensor_personas
            ascensor_lleno    = (len(ascensor_personas)==CAP_ASCENSOR)

            personas_esta_planta_suben = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]>ascensor_pos]
            personas_esta_planta_bajan = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]<ascensor_pos]


            # 1.- Si alguien ha llegado a su destino, o bien, si su destino está por encima
            #     o por debajo del rango y si se ha llegado al límite de este, entonces
            #     forzamos que baje del ascensor, devolviendo como unica accion posible DEJAR PERSONA. (Restriccion 2)
            for persona in ascensor_personas:
                if self.goal[persona]==ascensor_pos or \
                (self.goal[persona]<ascensor_rango[0] and ascensor_pos==ascensor_rango[0]) or \
                (self.goal[persona]>ascensor_rango[1] and ascensor_pos==ascensor_rango[1]):
                    return [(DEJAR_PERSONA,num_ascensor,persona)]

            # 2.- SUBIR:

            persona_menos_arriba = min([person for person in personas if ascensor_pos<person and person<=ascensor_rango[1] and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)
            
            # Si el ascensor está vacío sólo subimos si hay alguna persona por arriba 
            # y no hay gente en esta planta que quiera subir (Restriccion 3 y 4)
            if ascensor_vacio:
                if persona_menos_arriba!=NINGUNA and not(personas_esta_planta_suben):
                    accs.append((SUBIR,num_ascensor,persona_menos_arriba-ascensor_pos))

            # Si el ascensor no está vacío, comprobamos que sus personas suben y vamos
            # al mínimo entre a donde suben estas, donde hay una persona por encima y el rango superior
            # siempre que no nos quede espacio o no haya gente en esa planta que quiere subir
            # en cuyo caso cogeremos a dichar persona para acercarla lo maximo posible (Restriccion 4)
            elif self.goal[ascensor_personas[0]]>ascensor_pos:
                destino_menos_arriba = min(ascensor_rango[1], min([self.goal[i] for i in ascensor_personas]))

                if ascensor_lleno or not(personas_esta_planta_suben):
                    if persona_menos_arriba==NINGUNA:
                        accs.append((SUBIR,num_ascensor,destino_menos_arriba-ascensor_pos))
                    else:
                        accs.append((SUBIR,num_ascensor,min(persona_menos_arriba, destino_menos_arriba)-ascensor_pos))


            # 3.- BAJAR. La acción bajar es totalmente dual a la acción subir, mismas lógica en
            #     las condiciones.  

            persona_menos_abajo = max([person for person in personas if ascensor_pos>person and person>=ascensor_rango[0] and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo bajamos si hay alguna persona por abajo
            if ascensor_vacio:
                if persona_menos_abajo!=NINGUNA and not(personas_esta_planta_bajan):
                    accs.append((BAJAR,num_ascensor,ascensor_pos-persona_menos_abajo))
            elif self.goal[ascensor_personas[0]]<ascensor_pos:
                destino_menos_abajo = max(ascensor_rango[0], max([self.goal[i] for i in ascensor_personas]))

                if ascensor_lleno or not(personas_esta_planta_bajan):
                    if persona_menos_abajo==NINGUNA:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-destino_menos_abajo))
                    else:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-max(persona_menos_abajo, destino_menos_abajo)))

            # 4.- DEJAR PERSONA: Para cada persona del ascensor, podemos dejarla
            for personaInterior in ascensor_personas:
                accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))

            # 5. COGER PERSONA. En el interior de ascensor todos suben, o bajan, no ambas (Restriccion 1)
            if not ascensor_lleno:
                #Si el ascensor esta vacio, podemos coger a cualquiera, suban o bajen
                if ascensor_vacio:
                    if ascensor_pos != ascensor_rango[1]:
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    if ascensor_pos != ascensor_rango[0]:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                else: 
                    #si tiene a alguien dentro, ese alguien va hacia adentro del rango, si no forzábamos su salida en 1.-
                    #Si el ascensor tiene a alguien que baja, solo cogemos a gente que baje
                    if self.goal[ascensor_personas[0]]<ascensor_pos:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    else:
                    #Si el ascensor tiene a alguien que sube, solo cogemos a gente que suba
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))

        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        
        ascensor = action[1]
        
        #Si bajamos, la nueva posicion del ascensor será la actual menos las plantas a bajar
        #y el instante de llegada será el anterior más el nº de plantas recorridas
        if action[0]==BAJAR:
            estado_nuevo.ascensor_pos[ascensor] -= action[2]
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        #Si subimos, la nueva posicion del ascensor será la actual más las plantas a subir
        #y el instante de llegada será el anterior más el nº de plantas recorridas
        elif action[0]==SUBIR:
            estado_nuevo.ascensor_pos[ascensor] += action[2]
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += action[2]
        #Si cogemos persona, introducimos en ascensor_personas la persona que cogemos. Además indicamos en
        #personas que se encuentra EN_ASCENSOR.
        #También tenemos que tener en cuenta que, como la ha cogido, ambos deben estar en el mismo instante,
        #por lo que si el instante del ascensor era menor, marcamos la espera del ascensor actualizándolo al instante de la persona
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas[ascensor].append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
            estado_nuevo.ascensores_tiempo_llegada[ascensor] += max(state.personas_tiempo_llegada[action[2]]-state.ascensores_tiempo_llegada[ascensor],0)
        #Si dejamos persona, la quitamos de ascensor_personas. Si ha llegado a su destino, ponemos EN_DESTINO
        #y si no, la nueva planta en la que se encuentra (la del ascensor)
        #También, actualizamos su tiempo de llegada (usando el del ascensor, actualizado en la subida/bajada)
        else:
            estado_nuevo.ascensor_personas[ascensor].remove(action[2])
            estado_nuevo.personas_tiempo_llegada[action[2]] = state.ascensores_tiempo_llegada[ascensor]
       
            if self.goal[action[2]]==state.ascensor_pos[ascensor]:
                estado_nuevo.personas[action[2]] = EN_DESTINO
            else:
                estado_nuevo.personas[action[2]] = state.ascensor_pos[ascensor]
        
        return estado_nuevo
    
    def goal_test(self, state):
        self.analizados +=1
        return state.personas == [EN_DESTINO for _ in state.personas]

    # El coste es lo que ahora tarda a más el ascensor más lento (no necesariamente el mismo)
    def path_cost(self, c, state1, action, state2):
        return max(state2.ascensores_tiempo_llegada)-max(state1.ascensores_tiempo_llegada)
    
    # Heurística que garantiza solución óptima, eliminando la división por CAP_ASCENSOR*NUM_ASCENSORES
    # (número total de personas moviéndose a la vez) obtenemos una heurística que alcanza solución 
    # (aunque no necesariamente óptima) en un menor tiempo
    def h(self,node):
        suma = sum([abs(self.goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        for ascensor in range(0, len(node.state.ascensor_personas)):
            suma += sum([abs(self.goal[persona]-node.state.ascensor_pos[ascensor]) for persona in node.state.ascensor_personas[ascensor]])
        return suma/(CAP_ASCENSOR*NUM_ASCENSORES)
        
    def value(self, state):
        raise NotImplementedError

#### Probando nuestra versión final paralelizada
Inicializamos el nodo init con las plantas donde se encuentran al comienzo cada persona y los rangos de cada ascensor a considerar, y la lista goal con las plantas a las que quieren ir. Además, tenemos que actualizar las constantes convenientemente. Declaramos el problema con dichos nodos y luego ejecutamos la búsqueda.
Posteriormente, decodificamos la lista de acciones obtenidas <u>(Módulo SolutionManagerParalelo)</u> para visualizarla cómodamente con una interpretación y no como tuplas codificadas.

Vamos a probar los casos de prueba de la versión anterior pero esta vez usando rangos con los ascensores. Como podemos observar, logramos hallar una solución al problema para una gran cantidad de personas, varios ascensores y varios rangos en un tiempo muy disminuido.

In [75]:
#Caso sencillo

NUM_PLANTAS    = 4
NUM_ASCENSORES = 2
NUM_PERSONAS = 2
CAP_ASCENSOR   = 2

init = NodeState([2,0],[(0,3),(3,4)])
goal = [4,1]
problem = AscensoresParalelizados(init, goal)
start = time.time()
acciones = astar_search(problem).solution()
end = time.time()
acciones_ampliadas = ampliadas_acciones_paralelas(init,acciones)
print("Acciones de los ascensores")
print("Ordenadas por ascensor y momento en el que se realizan")
print()
decodificador_acciones_paralelas(cocina_acciones_paralelas(acciones_ampliadas))

Acciones de los ascensores
Ordenadas por ascensor y momento en el que se realizan

[0] -> El ascensor 0 coge a la persona 1
[0,1] -> El ascensor 0 sube 1 planta
[1] -> El ascensor 0 deja a la persona 1
[1,2] -> El ascensor 0 sube 1 planta
[2] -> El ascensor 0 coge a la persona 0
[2,3] -> El ascensor 0 sube 1 planta
[3] -> El ascensor 0 deja a la persona 0
[0,3] -> El ascensor 1 espera
[3] -> El ascensor 1 coge a la persona 0
[3,4] -> El ascensor 1 sube 1 planta
[4] -> El ascensor 1 deja a la persona 0


In [76]:
print("Nodos analizados: " + str(problem.analizados))
print("Coste secuencial: "+str(coste_acciones(acciones)))
print("Coste paralelo: "+str(coste_acciones_paralelas(init,acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 14
Coste secuencial: 4
Coste paralelo: 4
Tiempo de búsqueda: 0.000997304916381836 s


In [77]:
#Caso del enunciado

NUM_PLANTAS    = 12
NUM_ASCENSORES = 4
NUM_PERSONAS = 5
CAP_ASCENSOR   = 2

init = NodeState([2,4,1,8,1],[(0,4),(4,8),(4,8),(8,12)])
goal = [3,11,12,1,9]
problem = AscensoresParalelizados(init, goal)
start = time.time()
acciones = astar_search(problem).solution()
end = time.time()
acciones_ampliadas = ampliadas_acciones_paralelas(init,acciones)
print("Acciones de los ascensores")
print("Ordenadas por ascensor y momento en el que se realizan")
print()
decodificador_acciones_paralelas(cocina_acciones_paralelas(acciones_ampliadas))

Acciones de los ascensores
Ordenadas por ascensor y momento en el que se realizan

[0,1] -> El ascensor 0 sube 1 planta
[1] -> El ascensor 0 coge a las personas 4, 2
[1,2] -> El ascensor 0 sube 1 planta
[2] -> El ascensor 0 deja a la persona 4
[2] -> El ascensor 0 coge a la persona 0
[2,3] -> El ascensor 0 sube 1 planta
[3] -> El ascensor 0 deja a las personas 0, 2
[3,4] -> El ascensor 0 baja 1 planta
[4] -> El ascensor 0 coge a la persona 4
[4,5] -> El ascensor 0 sube 1 planta
[5] -> El ascensor 0 coge a la persona 2
[5,6] -> El ascensor 0 sube 1 planta
[6] -> El ascensor 0 deja a las personas 4, 2
[6,8] -> El ascensor 0 espera
[8] -> El ascensor 0 coge a la persona 3
[8,11] -> El ascensor 0 baja 3 plantas
[11] -> El ascensor 0 deja a la persona 3
[0] -> El ascensor 1 coge a la persona 1
[0,4] -> El ascensor 1 sube 4 plantas
[4] -> El ascensor 1 deja a la persona 1
[4] -> El ascensor 1 coge a la persona 3
[4,8] -> El ascensor 1 baja 4 plantas
[8] -> El ascensor 1 deja a la persona 3
[

In [78]:
print("Nodos analizados: "+str(problem.analizados))
print("Coste secuencial: "+str(coste_acciones(acciones)))
print("Coste paralelo: "+str(coste_acciones_paralelas(init,acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 1309
Coste secuencial: 35
Coste paralelo: 14
Tiempo de búsqueda: 3.482654571533203 s


**¡Resolvemos el caso del enunciado <u>óptimamente</u> en 3.26 segundos!**.

El coste para este mismo problema (tanto en nodos analizados como en tiempo) es menor que el obtenido en la versión anterior pese a que esta no considera la paralelización de los ascensores. Esto es así gracias a que con nuestra nueva función de coste favorecemos siempre las acciones en las que los ascensores están todos siendo utilizados. 

El cambio del paradigma, la pérdida de secuencialidad, hace que se analicen antes aquellas acciones que involucran varios ascensores simultaneamente. Una vez un ascensor esté funcionando, todas las acciones de los restantes ascensores que terminen de ejecutarse antes de que termine la acción de este ascensor tendrán coste cero, lo que gracias al algoritmo A\* implica que serán analizadas con preferencia frente a volver a hacer funcionar ese ascensor.

Distinguimos también entre coste secuencial (el número de plantas que recorren los ascensores del problema) y coste paralelo (número de plantas máximo que tiene que recorrer un ascensor). Gracias a que nuestra heurística es admisible obtenemos la solución óptima al problema planteado inicialmente.

#### Conclusiones

La definición de las acciones, muy restrictiva gracias a considerar los invariantes implícitos del problema, junto con el control de nodos repetidos del algoritmo, nos ha permitido conseguir una versión final del problema que nos lo solucione para una cantidad numerosa de personas, de ascensores y de bloques en un tiempo razonable.

<u>Además, el paralelizar los ascensores provoca que acabe muchísimo antes, lo que es una mejora considerable.</u>